In [ ]:
!pip install --upgrade torch datasets transformers torchaudio scikit-learn bitsandbytes accelerate -q
!pip install --upgrade torch datasets transformers torchaudio scikit-learn bitsandbytes accelerate -q



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201

In [ ]:
!pip install --upgrade torch torchvision torchaudio datasets transformers scikit-learn bitsandbytes accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.


In [ ]:
!pip uninstall -y transformers
!pip install transformers --upgrade --no-cache-dir


Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 80.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler
from torch.cuda.amp import autocast, GradScaler
from transformers import (
    DistilBertTokenizer, DistilBertForSequenceClassification,
    BartTokenizer, BartForSequenceClassification,
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments,
    Wav2Vec2ForCTC, Wav2Vec2Processor
)
from datasets import load_dataset, Dataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import torchaudio
import logging
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


GPU: Tesla T4


In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Cell 3: Load and subsample datasets
daily_dialog = load_dataset("daily_dialog")
woz_data = load_dataset("PolyAI/woz_dialogue", "en")
taskmaster_train = woz_data['train']


In [ ]:
taskmaster_train[:1]

{'dialogue_idx': [0],
 'dialogue': [[{'turn_label': [['food', 'eritrean']],
    'asr': [['Are there any eritrean restaurants in town?']],
    'system_transcript': '',
    'turn_idx': 0,
    'belief_state': [{'slots': [['food', 'eritrean']], 'act': 'inform'}],
    'transcript': 'Are there any eritrean restaurants in town?',
    'system_acts': []},
   {'turn_label': [['food', 'chinese']],
    'asr': [['How about Chinese food?']],
    'system_transcript': 'No, there are no eritrean restaurants in town. Would you like a different restaurant? ',
    'turn_idx': 1,
    'belief_state': [{'slots': [['food', 'chinese']], 'act': 'inform'}],
    'transcript': 'How about Chinese food?',
    'system_acts': []},
   {'turn_label': [['area', 'east']],
    'asr': [['I would like the East part of town.']],
    'system_transcript': 'There is a wide variety of Chinese restaurants, do you have an area preference or a price preference to narrow it down?',
    'turn_idx': 2,
    'belief_state': [{'slots': [[

In [ ]:
print(f"PolyAI/woz_dialogue train dialogues: {len(taskmaster_train)}")
print(f"DailyDialog train dialogues: {len(daily_dialog['train'])}")
print(f"Sample PolyAI/woz_dialogue keys: {list(taskmaster_train[0].keys())}")
print(f"Sample dialogue first turn: {taskmaster_train[0]['dialogue'][0]}")

PolyAI/woz_dialogue train dialogues: 600
DailyDialog train dialogues: 11118
Sample PolyAI/woz_dialogue keys: ['dialogue_idx', 'dialogue']
Sample dialogue first turn: {'turn_label': [['food', 'eritrean']], 'asr': [['Are there any eritrean restaurants in town?']], 'system_transcript': '', 'turn_idx': 0, 'belief_state': [{'slots': [['food', 'eritrean']], 'act': 'inform'}], 'transcript': 'Are there any eritrean restaurants in town?', 'system_acts': []}


In [ ]:

train_dialogs = daily_dialog['train'].shuffle(seed=42).select(range(int(0.5 * len(daily_dialog['train']))))
val_dialogs = daily_dialog['validation']

In [ ]:
def flatten_dialogues(dataset_split):
    utterances, acts, emotions = [], [], []
    for dialog in dataset_split:
        for utt, act, emo in zip(dialog['dialog'], dialog['act'], dialog['emotion']):
            if isinstance(utt, str) and utt.strip():
                utterances.append(utt)
                acts.append(act - 1)  # Map to 0-based
                emotions.append(emo)
            else:
                logger.warning(f"Invalid utterance skipped in DailyDialog: {utt}")
    return Dataset.from_dict({'text': utterances, 'label': acts, 'emotion': emotions})

train_data = flatten_dialogues(train_dialogs)
val_data = flatten_dialogues(val_dialogs)

**Note:**  
This implementation uses **BERT** for classification.  
Other models such as **Mistral** and **BART** were listed as options but have **not been tried** in this notebook.



In [ ]:
# Cell 5: Train dialogue act classifier (DistilBERT or BART)
use_bart = False  # Toggle for BART
use_mistral = False  # Toggle for Mistral zero-shot

In [ ]:
if use_mistral:
    tokenizer = AutoTokenizer.from_pretrained('mistralai/Mixtral-8x7B-Instruct-v0.1')
    model = AutoModelForCausalLM.from_pretrained(
        'mistralai/Mixtral-8x7B-Instruct-v0.1',
        load_in_4bit=True,
        device_map='auto'
    )
elif use_bart:
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
    model = BartForSequenceClassification.from_pretrained('facebook/bart-large', num_labels=4)
else:
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
if not use_mistral:
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True)

    tokenized_train = train_data.map(tokenize_function, batched=True)
    tokenized_val = val_data.map(tokenize_function, batched=True)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=16 if use_bart else 32,
        per_device_eval_batch_size=16 if use_bart else 32,
        eval_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        fp16=True,
        gradient_accumulation_steps=4 if use_bart else 2,
        learning_rate=2e-5,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
    )




Map:   0%|          | 0/43969 [00:00<?, ? examples/s]

Map:   0%|          | 0/8069 [00:00<?, ? examples/s]

In [ ]:
trainer.train()
model.save_pretrained('./dialog_act_model')
tokenizer.save_pretrained('./dialog_act_model')

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: notifyashishdeepak (notifyashishdeepak-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.588400,0.550758
2,0.465400,0.544782
3,0.402600,0.543216


('./dialog_act_model/tokenizer_config.json',
 './dialog_act_model/special_tokens_map.json',
 './dialog_act_model/vocab.txt',
 './dialog_act_model/added_tokens.json')

In [ ]:
directed_utterances = []
for dialogue in taskmaster_train.shuffle(seed=42):
    prev_utt = ""
    for turn in dialogue['dialogue']:
        user_text = turn.get('transcript', '')
        system_text = turn.get('system_transcript', '')
        if isinstance(user_text, str) and user_text.strip():
            directed_utterances.append((prev_utt, user_text, 1, 0))
        prev_utt = system_text.strip() if system_text.strip() else user_text.strip() or ""
        if not user_text.strip() and not system_text.strip():
            logger.warning(f"Invalid turn skipped: {turn}")

In [ ]:
non_directed_utterances = []
for dialog in train_dialogs:
    prev_utt = ""
    for utt, emo in zip(dialog['dialog'], dialog['emotion']):
        if isinstance(utt, str) and utt.strip():
            non_directed_utterances.append((prev_utt, utt, 0, emo))
            prev_utt = utt
        else:
            logger.warning(f"Invalid utterance skipped in DailyDialog: {utt}")
non_directed_utterances = non_directed_utterances[:len(directed_utterances)]

In [ ]:
all_utterances = directed_utterances + non_directed_utterances
texts = [(prev, curr) for prev, curr, _, _ in all_utterances]
emotions = [emo for _, _, _, emo in all_utterances]

In [ ]:

if not all(isinstance(prev, str) and isinstance(curr, str) and curr.strip() for prev, curr in texts):
    raise ValueError("All current utterances must be non-empty strings")

In [ ]:
logger.info(f"Total utterances: {len(all_utterances)}")
logger.info(f"Directed utterances: {len(directed_utterances)}")
logger.info(f"Sample directed utterances: {directed_utterances[:5]}")
logger.info(f"Non-directed utterances: {len(non_directed_utterances)}")
labels = [label for _, _, label, _ in all_utterances]
logger.info(f"Class distribution: Directed={sum(labels)}/{len(labels)}, Non-directed={len(labels)-sum(labels)}/{len(labels)}")


In [ ]:
print(f"Total utterances: {len(all_utterances)}")
print(f"Directed utterances: {len(directed_utterances)}")
print(f"Sample directed utterances: {directed_utterances[:5]}")
print(f"Non-directed utterances: {len(non_directed_utterances)}")
labels = [label for _, _, label, _ in all_utterances]
print(f"Class distribution: Directed={sum(labels)}/{len(labels)}, Non-directed={len(labels) - sum(labels)}/{len(labels)}")


Total utterances: 5072
Directed utterances: 2536
Sample directed utterances: [('', 'Are there any cheap restaurants in the south part of town? ', 1, 0), ('Are there any cheap restaurants in the south part of town?', 'I do not have a preference. ', 1, 0), ('There are two choices in the South area serving cheap food. Do you prefer Portuguese or Chinese?', 'Please give me the address and phone number for Nandos. ', 1, 0), ('Nandos is the Portuguese place, and The Lucky Star is Chinese.  Would you like more information on one or both of those?', 'No, thank you. Good bye.', 1, 0), ('', 'Looking for a restaurant in the East side of town', 1, 0)]
Non-directed utterances: 2536
Class distribution: Directed=2536/5072, Non-directed=2536/5072


In [ ]:
if not use_mistral:
    model = (BartForSequenceClassification if use_bart else DistilBertForSequenceClassification).from_pretrained('./dialog_act_model')
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

In [ ]:
class TextDataset(Dataset):
        def __init__(self, texts):
            if not isinstance(texts, list):
                raise TypeError("texts must be a list")
            self.texts = texts

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, idx):
            logger.debug(f"Accessing index: {idx}, type: {type(idx)}")
            if isinstance(idx, list):
                return [self.texts[i] for i in idx]
            if not isinstance(idx, int):
                raise TypeError("Index must be an integer")
            return self.texts[idx]


In [ ]:
  def collate_fn(batch):
        if any(isinstance(item, list) for item in batch):
            batch = [item for sublist in batch for item in sublist]
        batch = [item for item in batch if isinstance(item, tuple) and len(item) == 2 and isinstance(item[1], str) and item[1].strip()]
        if not batch:
            logger.error("Empty batch after filtering")
            return None
        combined_texts = [f"{prev.strip() + ' ' if prev.strip() else ''}[SEP] {curr}" for prev, curr in batch]
        return tokenizer(combined_texts, padding=True, truncation=True, return_tensors='pt')

In [ ]:
text_dataset = TextDataset(texts)
dataloader = DataLoader(
        texts,
        batch_size=16 if use_bart else 32,
        sampler=SequentialSampler(text_dataset),
        collate_fn=collate_fn,
    )


In [ ]:
predicted_acts, embeddings = [], []
with torch.no_grad():
    for batch in dataloader:
        if batch is None:
            continue
        inputs = {k: v.to(device) for k, v in batch.items()}
        with autocast():

            outputs = model(**inputs, output_hidden_states=True)

            if use_bart:
                 # For BART, we use the encoder's last hidden state for classification
                 cls_embeddings = outputs.encoder_last_hidden_state[:, 0, :]
            else:
                 # For DistilBERT, we use the last hidden state's CLS token embedding
                 cls_embeddings = outputs.hidden_states[-1][:, 0, :]

            logits = outputs.logits # Access logits directly from the model output
        embeddings.extend(cls_embeddings.cpu().numpy())
        preds = torch.argmax(logits, dim=1)
        predicted_acts.extend(preds.cpu().numpy())

# These lines should be outside the torch.no_grad() block but inside the main script flow
if len(embeddings) != len(all_utterances):
    # Add more context to the error message if possible
    logger.error(f"Embedding generation failed for some items. Expected {len(all_utterances)}, got {len(embeddings)}")
    # Optionally, inspect the data causing issues or filter them out
    # For now, we just raise the error
    raise ValueError(f"Embedding mismatch: got {len(embeddings)}, expected {len(all_utterances)}. Check data and batch processing.")

embeddings = np.array(embeddings)
predicted_acts = np.array(predicted_acts)
one_hot_acts = np.eye(4)[predicted_acts]
one_hot_emotions = np.eye(7)[emotions] # Assuming 'emotions' variable is available and corresponds to all_utterances
X = np.concatenate([embeddings, one_hot_acts, one_hot_emotions], axis=1)
y = np.array([label for _, _, label, _ in all_utterances]) # Assuming 'labels' are from all_utterances

# Continue with the rest of the code for splitting and training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<ipython-input-54-5e51e97b0937>:7: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


In [ ]:
if not use_mistral:
    class MainClassifier(nn.Module):
        def __init__(self, input_dim=768+4+7):
            super(MainClassifier, self).__init__()
            self.fc1 = nn.Linear(input_dim, 256)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.3)
            self.fc2 = nn.Linear(256, 128)
            self.fc3 = nn.Linear(128, 2)

        def forward(self, x):
            x = self.fc1(x)
            x = self.relu(x)
            x = self.dropout(x)
            x = self.fc2(x)
            x = self.relu(x)
            x = self.fc3(x)
            return x

    main_model = MainClassifier(input_dim=1024+4+7 if use_bart else 768+4+7).to(device)

In [ ]:
from torch.amp import autocast, GradScaler

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class_weights = torch.tensor(
    [1.0, 2.0 * len(y_train) / (sum(y_train) + 1e-6)], dtype=torch.float32
).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(main_model.parameters(), lr=2e-4)
scaler = GradScaler()

num_epochs = 5
for epoch in range(num_epochs):
    main_model.train()
    for batch in train_loader:
        inputs, labels = batch
        inputs = inputs.to(device, dtype=torch.float32)
        labels = labels.to(device)

        optimizer.zero_grad()
        with autocast(device_type='cuda'):
            outputs = main_model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    logger.info(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')


In [ ]:
# Cell 8: Evaluate main classifier
if not use_mistral:
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    main_model.eval()
    with torch.no_grad():
        # Specify the device_type for autocast
        with autocast(device_type='cuda'):
            outputs = main_model(X_test_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
            precision, recall, f1, _ = precision_recall_fscore_support(y_test_tensor.cpu(), predicted.cpu(), average='binary')
        logger.info(f'Test Accuracy: {accuracy:.4f}')
        logger.info(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')

In [ ]:
def predict_directed(text, prev_text="", use_bart=use_bart, use_mistral=use_mistral):
    if not isinstance(text, str) or not text.strip():
        raise ValueError("Input text must be a non-empty string")
    if not isinstance(prev_text, str):
        prev_text = ""
    if use_mistral:
        prompt = (
            f"Previous utterance: {prev_text}\n"
            f"Current utterance: {text}\n"
            f"Is this utterance directed to an AI (e.g., a command or question like 'Can you tell me the weather?')? "
            f"Answer 'Yes' or 'No'."
        )
        inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=10)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        logger.debug(f"Mistral response: {response}")
        return "Directed" if "Yes" in response else "Not directed"
    else:
        combined_text = f"{prev_text.strip() + ' ' if prev_text.strip() else ''}[SEP] {text}"
        inputs = tokenizer(combined_text, return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            with autocast():
                outputs = model(**inputs)
                cls_embedding = outputs.encoder_last_hidden_state[:, 0, :] if use_bart else outputs.last_hidden_state[:, 0, :]
                logits = model.classifier(cls_embedding)
                pred_act = torch.argmax(logits, dim=1).item()
        one_hot_act = np.eye(4)[pred_act]
        one_hot_emotion = np.eye(7)[0]
        feature = np.concatenate([cls_embedding.cpu().numpy(), one_hot_act[None, :], one_hot_emotion[None, :]], axis=1)
        feature_tensor = torch.tensor(feature, dtype=torch.float32).to(device)
        with torch.no_grad():
            with autocast():
                output = main_model(feature_tensor)
                _, prediction = torch.max(output, 1)
        logger.debug(f"Inference: text='{text}', prev_text='{prev_text}', label={prediction.item()}")
        return "Directed" if prediction.item() == 1 else "Not directed"

In [ ]:
def predict_directed(text, prev_text="", use_bart=use_bart, use_mistral=use_mistral):
    if not isinstance(text, str) or not text.strip():
        raise ValueError("Input text must be a non-empty string")
    if not isinstance(prev_text, str):
        prev_text = ""
    if use_mistral:
        prompt = (
            f"Previous utterance: {prev_text}\n"
            f"Current utterance: {text}\n"
            f"Is this utterance directed to an AI (e.g., a command or question like 'Can you tell me the weather?')? "
            f"Answer 'Yes' or 'No'."
        )
        inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=10)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        logger.debug(f"Mistral response: {response}")
        return "Directed" if "Yes" in response else "Not directed"
    else:
        combined_text = f"{prev_text.strip() + ' ' if prev_text.strip() else ''}[SEP] {text}"
        inputs = tokenizer(combined_text, return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            # Add device_type='cuda' to autocast
            with autocast(device_type='cuda'):
                # Explicitly request hidden states during the forward pass
                outputs = model(**inputs, output_hidden_states=True)

                # Handle potential differences in output attributes based on model type
                # DistilBERT uses 'last_hidden_state', BART uses 'encoder_last_hidden_state'
                if use_bart:
                     # For BART, we use the encoder's last hidden state for classification
                     # Ensure this is the correct attribute based on the specific BARTForSequenceClassification model
                     # Sometimes the output object structure can vary slightly
                     if hasattr(outputs, 'encoder_last_hidden_state'):
                         cls_embedding = outputs.encoder_last_hidden_state[:, 0, :]
                     elif hasattr(outputs, 'last_hidden_state'): # Fallback or alternative check
                          cls_embedding = outputs.last_hidden_state[:, 0, :]
                     else:
                          # If neither expected attribute is present, log and raise an error
                          logger.error("Model output does not have expected hidden state attribute for BART.")
                          logger.debug(f"Model outputs keys: {outputs.keys()}")
                          raise AttributeError("Model output does not have expected hidden state attribute for BART.")
                else:
                     # For DistilBERT, we use the last hidden state's CLS token embedding
                     # For DistilBERTForSequenceClassification, .hidden_states is a tuple
                     # The last element is the output of the last transformer layer
                     # The [:, 0, :] indexes the CLS token
                     # Check for hidden_states tuple first, then last_hidden_state
                     if hasattr(outputs, 'hidden_states') and isinstance(outputs.hidden_states, tuple) and len(outputs.hidden_states) > 0:
                         cls_embedding = outputs.hidden_states[-1][:, 0, :]
                     elif hasattr(outputs, 'last_hidden_state'): # Alternative check if hidden_states is not directly available
                         cls_embedding = outputs.last_hidden_state[:, 0, :]
                     else:
                         # If neither expected attribute is present, log and raise an error
                         logger.error("Model output does not have expected hidden state attribute for DistilBERT.")
                         logger.debug(f"Model outputs keys: {outputs.keys()}")
                         raise AttributeError("Model output does not have expected hidden state attribute for DistilBERT.")


                # Ensure the model object has a classifier head when not using Mistral
                if not hasattr(model, 'classifier'):
                     raise AttributeError("Model object does not have a 'classifier' attribute required for prediction.")
                # Use the model's internal classifier with the extracted CLS embedding
                logits = model.classifier(cls_embedding)
                pred_act = torch.argmax(logits, dim=1).item()

        # Ensure emotion is handled correctly. Assuming a default emotion (e.g., 0) if not available.


        one_hot_act = np.eye(4)[pred_act]
        # Use a default emotion since emotion is not predicted here
        # Assuming 0 is a valid default index for emotion
        one_hot_emotion = np.eye(7)[0] # Using 7 emotion classes and 0 as default index
        feature = np.concatenate([cls_embedding.cpu().numpy(), one_hot_act[None, :], one_hot_emotion[None, :]], axis=1)
        feature_tensor = torch.tensor(feature, dtype=torch.float32).to(device)

        with torch.no_grad():

             with autocast(device_type='cuda'):
                 # Ensure main_model exists and is on the correct device
                 if 'main_model' not in globals() or main_model is None:
                      raise NameError("main_model is not defined or loaded.")
                 main_model.eval() # Ensure main_model is in evaluation mode
                 output = main_model(feature_tensor)
                 _, prediction = torch.max(output, 1)
        logger.debug(f"Inference: text='{text}', prev_text='{prev_text}', label={prediction.item()}")
        return "Directed" if prediction.item() == 1 else "Not directed"

In [ ]:
examples = [
    ("What's the time?", "Can you tell me the weather?"),
    ("No, there are no eritrean restaurants in town.", "How about Chinese food?"),
    ("", "Could I get the address of Yu Garden?"),
]
for prev_text, text in examples:
    result = predict_directed(text, prev_text)
    print(f"Text: '{text}', Prev: '{prev_text}', Prediction: {result}")

Text: 'Can you tell me the weather?', Prev: 'What's the time?', Prediction: Directed
Text: 'How about Chinese food?', Prev: 'No, there are no eritrean restaurants in town.', Prediction: Directed
Text: 'Could I get the address of Yu Garden?', Prev: '', Prediction: Directed


In [ ]:
non_directed_examples = [
    ("I'm so tired after work.", "Yeah, I just want to relax tonight."),  # Casual conversation
    ("Did you watch the game last night?", "It was amazing!"),  # Emotional reaction
    ("The weather’s nice today.", "I’m thinking of going for a walk."),  # Descriptive statement
    ("", "I had a rough day at school."),  # Emotional expression, no context
    ("This movie is boring.", "I should’ve picked a different one."),  # Opinion sharing
]
for prev_text, text in non_directed_examples:
    result = predict_directed(text, prev_text)
    print(f"Text: '{text}', Prev: '{prev_text}', Prediction: {result}")

Text: 'Yeah, I just want to relax tonight.', Prev: 'I'm so tired after work.', Prediction: Not directed
Text: 'It was amazing!', Prev: 'Did you watch the game last night?', Prediction: Not directed
Text: 'I’m thinking of going for a walk.', Prev: 'The weather’s nice today.', Prediction: Not directed
Text: 'I had a rough day at school.', Prev: '', Prediction: Not directed
Text: 'I should’ve picked a different one.', Prev: 'This movie is boring.', Prediction: Not directed


In [ ]:
# prompt: download the trained model

import os
from google.colab import files

# Save the trained main_model
main_model_save_path = './main_classifier_model.pth'
torch.save(main_model.state_dict(), main_model_save_path)

# Save the trained dialog act model (DistilBERT/BART)
dialog_act_model_dir = './dialog_act_model'

# Package the models into a zip file for easier download
!zip -r /content/trained_models.zip {main_model_save_path} {dialog_act_model_dir}

# Download the zip file
files.download('/content/trained_models.zip')

  adding: main_classifier_model.pth (deflated 8%)
  adding: dialog_act_model/ (stored 0%)
  adding: dialog_act_model/model.safetensors (deflated 8%)
  adding: dialog_act_model/tokenizer_config.json (deflated 75%)
  adding: dialog_act_model/vocab.txt (deflated 53%)
  adding: dialog_act_model/special_tokens_map.json (deflated 42%)
  adding: dialog_act_model/config.json (deflated 50%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
text = "Can you tell me the weather?"
prev_text = "What's the time?"
print(predict_directed(text, prev_text))
text = "How about Chinese food?"
prev_text = "No, there are no eritrean restaurants in town."
print(predict_directed(text, prev_text))

Directed
Directed


**Note:**  
The audio processing component is still being explored.  
We are currently experimenting with **Wav2Vec** for speech-to-text and emotion recognition from audio, but it is not fully integrated into the pipeline yet.



In [ ]:
# Cell 10: Inference for audio input
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
wav2vec_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

def transcribe_audio(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(sample_rate, 16000)(waveform)
    inputs = processor(waveform.squeeze().numpy(), return_tensors="pt", sampling_rate=16000).input_values
    with torch.no_grad():
        logits = wav2vec_model(inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

In [ ]:
# Example usage
# audio_path = 'path/to/audio.wav'
# text = transcribe_audio(audio_path)
# print(predict_directed(text))